In [ ]:
!python --version
print("---")

# install and import modules
%pip install rdflib

import google
import IPython
import json
import pandas as pd
import rdflib
import os
import tabulate
import typing

# mount drive here to read files from the folder "My Drive > Colab_Notebooks > Formal_Ontology_of_Mathematics > creativity"
google.colab.drive.mount('/content/drive')

os.chdir("/content/drive/My Drive/Colab_Notebooks/Formal_Ontology_of_Mathematics/creativity")

print("---")
!pwd

print("---")
!ls

In [ ]:
import modules.queries as queries

In [ ]:
# parameters
file_name = "ontology_output_v3.ttl"

# general functions
def access_graph(file_name: str,
                 folder_name: str = "input") -> rdflib.Graph:
    input_file = os.path.join(folder_name, file_name)
    return rdflib.Graph().parse(input_file)

def sparql_to_df(kg: rdflib.Graph,
                 sparql_query: str):
    raw = kg.query(sparql_query)
    variables = raw.vars
    records = [{str(variables[i]): str(item) for i, item in enumerate(row)} for row in raw]
    records_df = pd.DataFrame(records)
    if "links" in records_df.columns:
        records_df["links"] = records_df["links"].astype(int)
    return records_df

def sparql_to_concat_df(kg: rdflib.Graph,
                        sparql_queries: list,
                        hebb: bool = False):
    if hebb:
        df = pd.concat(
            [sparql_to_df(kg, sparql_query) for sparql_query in sparql_queries],
            ignore_index = True).groupby(by=["o1", "o2"])["links"].sum().reset_index()
    else:
        df = pd.concat(
            [sparql_to_df(kg, sparql_query) for sparql_query in sparql_queries],
            ignore_index = True).groupby(by=["o"])["links"].sum().reset_index()
    return df

In [ ]:
def create_iris_for_values(proposition_number: int):
    iris_strings = [f"<https://www.foom.com/core#proof_{i}> <https://www.foom.com/core#proposition_{i}>" for i in range(1, proposition_number)]
    return " ".join(iris_strings)


def history(kg: rdflib.Graph,
            proposition_number: int = 0,
            base_sparql_queries: list = [
                [queries.direct_definitions(), queries.direct_postulates(), queries.direct_common_notions()],
                [queries.hierarchical_definitions(), queries.hierarchical_postulates(), queries.hierarchical_common_notions()],
                [queries.mereological_definitions(), queries.mereological_postulates(), queries.mereological_common_notions()] ],
            weights: list = [6/9, 1/9, 2/9]):
    query_sets = base_sparql_queries.copy()
    if proposition_number >= 2:
        # Generate the iris strings
        iris = create_iris_for_values(proposition_number)

        # Append the new queries to the existing lists
        query_sets[0].append(queries.direct_template_propositions_proofs(iris))
        query_sets[1].append(queries.hierarchical_template_propositions_proofs(iris))
        query_sets[2].append(queries.mereological_template_propositions_proofs(iris))

    # Generate the histories
    histories = [sparql_to_concat_df(kg, query_set) for query_set in query_sets]

    activation_dfs = []
    # calculation of activation potentials
    for history_df, weight in zip(histories, weights):
        total_use = history_df["links"].sum()
        actions_df = history_df.assign(
            activation_potential = (history_df["links"] * weight) / total_use
        )[["o", "activation_potential"]]
        activation_dfs.append(actions_df)

    # combine dataframes
    combined_df = pd.concat(activation_dfs, ignore_index=True)
    return combined_df.groupby("o")["activation_potential"].sum().reset_index()

def hebb(kg: rdflib.Graph,
         proposition_number: int = 0,
         sparql_queries: list = [queries.hebb_definitions(), queries.hebb_postulates(), queries.hebb_common_notions()]):
    if proposition_number >= 2:
        # Generate the iris strings
        iris = create_iris_for_values(proposition_number)
        # Append the new queries to the existing lists
        sparql_queries.append(queries.hebb_template_propositions_proofs(iris))
    df = sparql_to_concat_df(kg, sparql_queries, hebb=True)
    total_use = df["links"].sum()
    df["activation_potential"] = df["links"] / total_use
    df = df.drop(columns=["links"])
    df = df.sort_values(by="activation_potential", ascending=False)
    df = df.reset_index(drop=True)
    return df

# def calculate_preface_potentials(kg: rdflib.Graph):
def calculate_activation_potential(kg: rdflib.Graph,
                                   proposition_number: int = 0):
    # history potential
    history_potential = history(kg, proposition_number)
    print(len(history_potential))
    print(history_potential[:20])
    print(history_potential["activation_potential"].sum())

    # hebb potential
    hebb_potential = hebb(kg, proposition_number)
    print(len(hebb_potential))
    print(hebb_potential[:20])
    print(hebb_potential["activation_potential"].sum())

    return [], []

In [ ]:
"""
1. capture of last proposition: direct and mereological [TO DO]
2. capture of history and hebbian of previous propositions and proofs [DONE]
3. include history and hebbian of previous propositions and proofs in table [DONE]
4. prepare table of activation potential [DONE]
5. capture last proof: direct
6. check last proof against lowest quartile  of activation potential table with last propostion removed
"""

def main(file_name: str = file_name,
         proposition_number: int = 1):
    # access turtle file and put content in the kg (rdflib.Graph)
    kg = access_graph(file_name)
    #  calculate activation potential
    preface_history, preface_cooccurrence = calculate_activation_potential(kg, proposition_number)


    # check surprise score

    return

main(proposition_number=4)